In [35]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from copy import deepcopy
directory = "./result"
file_name = 'rdkit.npz'
random_seed = 2024
file_path = os.path.join(directory, file_name)

rdkit_npz = np.load(file_path, allow_pickle=True)

print("Arrays in the NPZ file:", PhO_rdkit_npz.files)
rdkit_data = rdkit_npz['data']
rdkit_columns = rdkit_npz['columns']
physorg_react_data_df = pd.DataFrame(rdkit_data, columns=rdkit_columns)

physorg_react_data_df.dropna(inplace=True)  


test_index = [87, 381, 316, 120, 479, 448, 197, 507, 198, 379, 508, 313, 265, 449, 453, 89, 292, 88, 121, 266, 454, 455, 447, 456, 271, 
              269, 294, 293, 297, 248, 249, 464, 312, 499, 289, 509, 511, 267, 166, 386, 290, 392, 498, 91, 488, 7, 278, 54, 285, 94, 303, 
              358, 444, 5, 286, 98, 458, 2, 291, 268, 470, 262, 308, 272, 53, 354, 512, 360, 495, 468, 391, 310, 307, 457, 306, 258, 352, 
              305, 357, 131, 362, 42, 311, 460, 304, 510, 298, 263, 350, 368, 100, 115, 61, 500, 56, 90, 442, 277, 487, 309, 0, 1, 356]

External_index = [462,119 ,396, 273, 375, 503, 318, 504, 472, 463, 465, 341, 342, 340, 477, 327, 348, 476,423 ,377, 332, 333, 182, 181,
                  349, 183, 112, 328, 323, 324, 113, 329, 330, 331, 144, 145,149, 150, 153, 151, 152, 154, 147, 485, 148, 482, 370, 373,
                  372, 146, 371]
length_db = len(physorg_react_data_df) // 2
            
test_index_db2 = [i + length_db for i in test_index]
External_index_db2 = [i + length_db for i in External_index]
all_indices = set(range(len(physorg_react_data_df)))  
train_val_indices = sorted(list(all_indices - set(test_index) - set(test_index_db2) - set(External_index) - set(External_index_db2)))  
X = physorg_react_data_df.drop(columns=['YRed/Ox', 'compound_name'])
y = physorg_react_data_df['YRed/Ox']
X_train_val = X.iloc[train_val_indices]
y_train_val = y.iloc[train_val_indices]
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val)

Arrays in the NPZ file: ['data', 'columns']


# Fine screening of hyperparameters and mapping of learning curves

In [36]:
model = ExtraTreesRegressor(n_jobs=-1,random_state=random_seed)

param_grid = {
    'n_estimators': [75,100,125],
    'max_depth': [10,15,20],
    'min_samples_split': [2,4,5,6],
    'min_samples_leaf': [1, 2, 3]
}
kfold = KFold(n_splits=10, shuffle=True, random_state=random_seed)

grid_search = GridSearchCV(model, param_grid, scoring='r2', cv=kfold, verbose=2, n_jobs=-1)

grid_search.fit(X_train_val_scaled, y_train_val)

print("Best parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
Best parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 125}


In [37]:
rates = [int(i) for i in np.linspace(0.1, 0.9, 20) * X_train_val_scaled.shape[0]]
print(rates)

r2_list = []
r2_std_list = []
r2_train_list = []
r2_train_std_list = []

np.random.seed(random_seed)
model = ExtraTreesRegressor(n_jobs=-1, min_samples_split=2, min_samples_leaf=1, n_estimators=125, max_depth=15, random_state=random_seed)
kf = KFold(n_splits=10, random_state=random_seed, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X_train_val_scaled, y_train_val, test_size=0.2, random_state=random_seed)
for rate in rates:
    r2s = []
    r2ts = []
    models = []
    y_pred = []
    maes = []
    rmses = []
    for train_index, val_index in kf.split(y_train):
        try:
            X_train_, X_test_, y_train_, y_test_ = train_test_split(
                X_train, y_train, train_size=rate)
        except:
            X_train_, y_train_ = X_train, y_train

        model.fit(X_train_, y_train_)
        models.append(deepcopy(model))
        y_pred.append(model.predict(X_test))
        valid_p, valid_y = model.predict(X_test), y_test
        r2 = r2_score(valid_y, valid_p)
        r2t = r2_score(model.predict(X_train_), y_train_)
        mae = mean_absolute_error(valid_y, valid_p)
        rmse = np.sqrt(mean_squared_error(valid_y, valid_p))
        r2s.append(r2)
        r2ts.append(r2t)
        maes.append(mae)
        rmses.append(rmse)
    
    valid_p, valid_y = np.array(y_pred).mean(axis=0), y_test
    r2 = np.array(r2s).mean(axis=0)
    r2_std = pd.DataFrame(r2s).std()[0]
    # print(r2_std)
    r2t = np.array(r2ts).mean(axis=0)
    r2t_std = pd.DataFrame(r2ts).std()[0]
    r2_list.append(r2)
    r2_train_list.append(r2t)
    r2_std_list.append(r2_std)
    r2_train_std_list.append(r2t_std)
    print(' $R^2$: {:.3}\n r2t: {:.3}\n r2std: {:.3}\n r2tstd: {:.3}'.format(
        r2, r2t, r2_std, r2t_std))
    
learning_curve = pd.DataFrame(columns=('r2', 'r2t', 'r2_std', 'r2t_std', 'num'))
learning_curve['r2'] = r2_list
learning_curve['r2t'] = r2_train_list
learning_curve['r2_std'] = r2_std_list
learning_curve['r2t_std'] = r2_train_std_list
learning_curve['num'] = rates
# learning_curve.to_csv('learn_curve.csv')
learning_curve

[73, 103, 134, 165, 195, 226, 257, 288, 318, 349, 380, 411, 441, 472, 503, 534, 564, 595, 626, 657]
 $R^2$: 0.951
 r2t: 1.0
 r2std: 0.00805
 r2tstd: 3.37e-08
 $R^2$: 0.959
 r2t: 1.0
 r2std: 0.00592
 r2tstd: 1.62e-07
 $R^2$: 0.961
 r2t: 1.0
 r2std: 0.00371
 r2tstd: 1.22e-06
 $R^2$: 0.959
 r2t: 1.0
 r2std: 0.00698
 r2tstd: 4.66e-06
 $R^2$: 0.963
 r2t: 1.0
 r2std: 0.00448
 r2tstd: 1.06e-05
 $R^2$: 0.969
 r2t: 1.0
 r2std: 0.00272
 r2tstd: 6.25e-06
 $R^2$: 0.971
 r2t: 1.0
 r2std: 0.00336
 r2tstd: 1.09e-05
 $R^2$: 0.971
 r2t: 1.0
 r2std: 0.00287
 r2tstd: 1.76e-05
 $R^2$: 0.972
 r2t: 1.0
 r2std: 0.00246
 r2tstd: 1.51e-05
 $R^2$: 0.974
 r2t: 1.0
 r2std: 0.00315
 r2tstd: 1.83e-05
 $R^2$: 0.976
 r2t: 1.0
 r2std: 0.00222
 r2tstd: 2.23e-05
 $R^2$: 0.976
 r2t: 1.0
 r2std: 0.00228
 r2tstd: 1.68e-05
 $R^2$: 0.976
 r2t: 1.0
 r2std: 0.00213
 r2tstd: 3.71e-05
 $R^2$: 0.978
 r2t: 1.0
 r2std: 0.00135
 r2tstd: 2.6e-05
 $R^2$: 0.978
 r2t: 1.0
 r2std: 0.00255
 r2tstd: 2.46e-05
 $R^2$: 0.979
 r2t: 1.0
 r2std:

,r2,r2t,r2_std,r2t_std,num
0,0.950723,1.000000,8.050886e-03,3.368208e-08,73
1,0.958625,1.000000,5.915117e-03,1.622158e-07,103
2,0.961492,0.999999,3.708076e-03,1.218838e-06,134
3,0.958572,0.999997,6.982872e-03,4.663966e-06,165
4,0.963396,0.999992,4.483043e-03,1.059521e-05,195
5,0.968730,0.999991,2.717888e-03,6.248809e-06,226
6,0.970755,0.999985,3.362543e-03,1.091725e-05,257
7,0.970763,0.999962,2.873704e-03,1.760118e-05,288
8,0.972089,0.999971,2.459971e-03,1.509006e-05,318
9,0.973796,0.999957,3.147577e-03,1.834364e-05,349


In [38]:
labels = rates  

min_label = min(labels)
max_label = max(labels)
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
r2 = learning_curve['r2']
r2t = learning_curve['r2t']
r2std = learning_curve['r2_std']
r2tstd = learning_curve['r2t_std']
labels = rates

xnew = np.linspace(min_label, max_label, 500)
func_1 = interp1d(labels, r2, kind='cubic')
r2_new = func_1(xnew)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
ax.plot(xnew,r2_new, '-', label = 'Cross-validation score')  
# ax.plot(labels, r2, '-', label = 'Cross-validation score')
ax.scatter(labels, r2, s=10)
ax.fill_between(labels, r2-r2std, r2+r2std, alpha=0.2, facecolor='b')


ax.plot(labels, r2t, '-r', label = 'Training score')
ax.scatter(labels, r2t, s=10, c='r')
ax.fill_between(labels, r2t-r2tstd, r2t+r2tstd, alpha=0.2, facecolor='r')
plt.xlim((0, 800))
plt.legend(loc='lower right')

ax.set_xlabel("Training examples", fontsize=12)
ax.set_ylabel("Score($R^2$)", fontsize=12)
ax.grid()
plt.savefig("./result/learning curve-rdkit.svg",dpi = 600)
plt.savefig('./result/learning curve-rdkit.png', dpi=600)

# Feature Engineering - Feature Screening

In [39]:
'''
The dataset is divided into training/validation and out-of-sample (OOS) test sets based on predefined indices. 
The division process ensures that the test data is completely held out during the training and validation phases 
to properly evaluate the model's performance on unseen data.

- `test_index`: This is the primary set of indices used to define the initial part of the OOS test set.
- `test_index_db2`: Additional test indices derived by adding half the length of the dataset (`length_db`) to each index in `test_index`.
  This approach accounts for the potential scenario where test data points are distributed across the dataset.

The following steps are executed in the code:
1. `length_db` is calculated as half the total number of data points, used to adjust the indices for the second half of the dataset.
2. `test_index_db2` is generated to cover any test data points that might appear in the latter half of the dataset.
3. `full_test_index` combines `test_index` and `test_index_db2` to form the complete set of indices for the OOS test set.
4. The data points at these indices are then extracted as the OOS test set (`oos_x`, `oos_y`), ensuring they are not used in model training or validation.
5. The remaining data points are used to create the training/validation set (`X_scaled`, `y_train_val`), which is also standardized to aid in model training.
'''

directory = "./result"
file_name = 'rdkit.npz'

file_path = os.path.join(directory, file_name)

rdkit_npz = np.load(file_path, allow_pickle=True)

print("Arrays in the NPZ file:", PhO_rdkit_npz.files)
rdkit_data = rdkit_npz['data']
rdkit_columns = rdkit_npz['columns']
physorg_react_data_df = pd.DataFrame(rdkit_data, columns=rdkit_columns)

physorg_react_data_df.dropna(inplace=True)  

test_index = [87, 381, 316, 120, 479, 448, 197, 507, 198, 379, 508, 313, 265, 449, 453, 89, 292, 88, 121, 266, 454, 455, 447, 456, 271, 
              269, 294, 293, 297, 248, 249, 464, 312, 499, 289, 509, 511, 267, 166, 386, 290, 392, 498, 91, 488, 7, 278, 54, 285, 94, 303, 
              358, 444, 5, 286, 98, 458, 2, 291, 268, 470, 262, 308, 272, 53, 354, 512, 360, 495, 468, 391, 310, 307, 457, 306, 258, 352, 
              305, 357, 131, 362, 42, 311, 460, 304, 510, 298, 263, 350, 368, 100, 115, 61, 500, 56, 90, 442, 277, 487, 309, 0, 1, 356]

External_index = [462,119 ,396, 273, 375, 503, 318, 504, 472, 463, 465, 341, 342, 340, 477, 327, 348, 476,423 ,377, 332, 333, 182, 181,
                  349, 183, 112, 328, 323, 324, 113, 329, 330, 331, 144, 145,149, 150, 153, 151, 152, 154, 147, 485, 148, 482, 370, 373,
                  372, 146, 371]
length_db = len(physorg_react_data_df) // 2
            
test_index_db2 = [i + length_db for i in test_index]
External_index_db2 = [i + length_db for i in External_index]
all_indices = set(range(len(physorg_react_data_df))) 
train_val_indices = sorted(list(all_indices - set(test_index) - set(test_index_db2) - set(External_index) - set(External_index_db2)))  
full_test_index = sorted(list(set(test_index + test_index_db2)))
X = physorg_react_data_df.drop(columns=['YRed/Ox', 'compound_name'])
y = physorg_react_data_df['YRed/Ox']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.iloc[train_val_indices])
y_train_val = y.iloc[train_val_indices]
test_x = scaler.transform(X.iloc[full_test_index])
test_y = y.iloc[full_test_index]

Arrays in the NPZ file: ['data', 'columns']


In [40]:
random_seed = 2024
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
# The max_depth and n_estimators are identified by hyperparameters optimization procedure, which is demonstrated in Notebook 3.
model = ExtraTreesRegressor(n_jobs=-1, min_samples_split=2, min_samples_leaf=1, n_estimators=125, max_depth=15, random_state=random_seed)
cv = KFold(n_splits=10, shuffle=True, random_state=random_seed)
selector = RFECV(model, step=1, min_features_to_select=1,cv=cv, n_jobs=-1)
selector = selector.fit(X_train_val_scaled, y_train_val)
sel_index = np.where(selector.support_==True)[0]
print('----Reserved Descriptors----')
print('Size: %d'%len(sel_index))
print('Index of selected descriptors', sel_index + 1)

----Reserved Descriptors----
Size: 86
Index of selected descriptors [  1   2   4   5   6  12  13  14  15  16  17  18  19  20  30  32  36  37
  39  40  41  42  44  45  46  47  48  49  50  52  53  54  56  57  60  62
  63  64  65  66  67  69  71  73  75  76  77  79  80  82  83  85  86  87
  88  89  90  92  93  94  95  96  97 100 101 103 104 105 106 107 109 110
 113 114 122 127 128 131 132 143 151 161 168 169 172 190]
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=4, n_estimators=75; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.8s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=7

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.5s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=6, n_estimators=75; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=6, n_estimators=125; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=6, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=4, n_estimators=75; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=100; total time=

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=125; total time=   1.4s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=125; total time=   1.3s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=75; total time=   0.7s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=4, n_estimators=75; total time=   0.8s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=125; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=5, n_estimators=75; total time=   0.5s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=6, n_estimators=100; total time=   0.5s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=2, n_estimators=100; total time= 

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=125; total time=   1.4s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=6, n_estimators=125; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=125; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=4, n_estimators=125; total time=   1.4s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=5, n_estimators=75; total time=   0.6s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=6, n_estimators=100; total time=   1.4s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=5, n_estimators=125; total time=   0.9s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=4, n_estimators=75; total time=

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=75; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=125; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=4, n_estimators=125; total time=   1.5s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=75; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=75; total time=   0.4s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=2, n_estimators=75; total time=   0.5s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=6, n_estimators=100; total time=  

...

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=6, n_estimators=125; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=6, n_estimators=75; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=75; total time=   0.4s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=100; total time=   0.6s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=4, n_estimators=75; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=125; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=75; total time=   1.1s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=6, n_estimators=125; total time=  

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=4, n_estimators=125; total time=   1.7s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=4, n_estimators=75; total time=   1.1s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=75; total time=   0.5s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=4, n_estimators=100; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.2s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=75; total time=   0.7s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=5, n_estimators=125; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=4, n_estimators=75; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=6, n_estimators=100; total time=  

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.2s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=4, n_estimators=125; total time=   1.3s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=4, n_estimators=100; total time=   0.8s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=125; total time=   1.5s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=125; total time=   1.4s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=5, n_estimators=75; total time=   1.2s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=2, n_estimators=125; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=6, n_estimators=100; total time=   0.8s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=75; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=125; total time=   1.3s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=75; total time=   0.8s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=75; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=75; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=6, n_estimators=125; total time=   1.0s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=75; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=6, n_estimators=125; total time=   1.3s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=5, n_estimators=125; total time=   1.4s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=5, n_estimators=75; total time=   0

[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   1.0s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=6, n_estimators=125; total time=   1.3s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=6, n_estimators=75; total time=   0.6s
[CV] END max_depth=10, min_samples_leaf=3, min_samples_split=2, n_estimators=125; total time=   1.2s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=6, n_estimators=75; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=2, min_samples_split=4, n_estimators=100; total time=   1.4s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=2, n_estimators=75; total time=   0.4s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=4, n_estimators=100; total time=   0.9s
[CV] END max_depth=15, min_samples_leaf=3, min_samples_split=6, n_estimators=125; total time= 

In [41]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 6))  

y_r2 = selector.cv_results_['mean_test_score']
x_1_index = [item + 1 for item in range(len(y_r2))]

max_index = np.argmax(y_r2) + 1
max_score = np.max(y_r2)

plt.plot(x_1_index, y_r2, c='royalblue')

plt.plot([max_index, max_index], [np.min(y_r2), max_score], c='lightgreen', linewidth=2.5, linestyle='--')

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.text(max_index + 1, max_score - 0.002, f'Max Score: {max_score:.3f}', fontsize=15, verticalalignment='top')
plt.text(max_index + 1, max_score - 0.005, f'Feature Number: {max_index}', fontsize=15, verticalalignment='top')

plt.xlabel('Feature Number', fontsize=18)
plt.ylabel('Score', fontsize=18)

plt.tick_params(axis='both', which='both', direction='in', bottom=True, left=True)

plt.tight_layout()
plt.savefig('./result/RFECV-rdkit.png', dpi=600)
plt.show()


# Model Training

Training set

In [42]:
from scipy.stats import pearsonr
val_Y = []
val_P = []
kfold = KFold(n_splits=10, shuffle=True, random_state=random_seed)
for train_idx,val_idx in kfold.split(X_train_val_scaled):
    train_x,val_x = X_scaled[:,sel_index][train_idx],X_scaled[:,sel_index][val_idx]
    train_y,val_y = y_train_val.iloc[train_idx],y_train_val.iloc[val_idx]
    val_P_ = []
    for try_ in range(10): 
        model.fit(train_x,train_y)
        val_p = model.predict(val_x)
        val_P_.append(val_p)
    val_P_ = np.mean(val_P_,axis=0)
    val_P.append(val_P_)
    val_Y.append(val_y)
val_P = np.concatenate(val_P)
val_Y = np.concatenate(val_Y)
mae = mean_absolute_error(val_Y,val_P)
r2 = r2_score(val_Y,val_P)
pearson_r,_ = pearsonr(val_Y,val_P)

print("MAE: %.4f, R2: %.4f, Pearson R: %.4f"%(mae,r2,pearson_r))

MAE: 0.1206, R2: 0.9792, Pearson R: 0.9895


In [55]:
plt.figure(figsize=(4.5,4.5))

plt.scatter(val_Y,val_P,c='royalblue')

plt.text(-3.2,2.6,'$R^2$: %.3f'%r2_score(val_Y,val_P),fontsize=16)
plt.text(-3.2,2.10,'Pearson R: %.3f'%pearsonr(val_Y,val_P)[0],fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%mean_absolute_error(val_Y,val_P),fontsize=16)

plt.plot([y.min(),y.max()],[y.min(),y.max()],c='royalblue')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)
plt.xticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=14)
plt.yticks([-3, -2, -1,0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=14)

plt.tick_params(bottom='on',left='on')
plt.tight_layout()
plt.savefig('./result/train-Grdkit.tif', dpi=600)  
plt.savefig('./result/train-Grdkit.png',dpi =600)

In [58]:
plt.figure(figsize=(4.5,4.5))

plt.scatter(val_Y,val_P,c='royalblue')

plt.text(-3.2,2.6,'$R^2$: %.3f'%r2_score(val_Y,val_P),fontsize=16)
plt.text(-3.2,2.10,'Pearson R: %.3f'%pearsonr(val_Y,val_P)[0],fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%mean_absolute_error(val_Y,val_P),fontsize=16)

plt.plot([y.min(),y.max()],[y.min(),y.max()],c='royalblue')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)
plt.xticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)
plt.yticks([-3, -2, -1,0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)

re_train = np.abs(val_P - val_Y) / np.abs(val_Y)
accuracy_train = np.mean(re_train <= 0.3) * 100
accuracy_train1 = np.mean(re_train <= 0.2) * 100
plt.text(-0.4, -2.4, f'RE±0.3: {accuracy_train:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold') 
plt.text(-0.4, -2.9, f'RE±0.2: {accuracy_train1:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold')

print(f"Training Accuracy (RE±0.3): {accuracy_train:.2f}%")
print(f"Training Accuracy (RE±0.2): {accuracy_train1:.2f}%")
plt.tick_params(bottom='on',left='on')
plt.tight_layout()
plt.savefig('./result/train-Grdkit-RE.tif', dpi=600)  
plt.savefig('./result/train-Grdkit-RE.png',dpi =600)

Training Accuracy (RE±0.3): 91.23%
Training Accuracy (RE±0.2): 85.89%


Test set

In [44]:
test_P = []
feature_importance = []
for _ in range(10):
    model.fit(X_scaled[:,sel_index],y_train_val)
    feature_importance.append(model.feature_importances_)
    test_p = model.predict(test_x[:,sel_index])
    test_P.append(test_p)
test_p = np.mean(test_P,axis=0)
feature_importance = np.mean(feature_importance,axis=0)
sel_feature_names = np.array(PhO_rdkit_columns)[sel_index + 1]
sorted_feature_index = np.argsort(feature_importance)
importance_desc_names = sel_feature_names[sorted_feature_index]
importance_of_sel_desc = feature_importance[sorted_feature_index]

r2 = r2_score(test_y,test_p)
pearson_r,_ = pearsonr(test_y,test_p)
mae = mean_absolute_error(test_y,test_p)
print("MAE: %.4f, R2: %.4f, Pearson R: %.4f"%(mae,r2,pearson_r))

MAE: 0.2548, R2: 0.9289, Pearson R: 0.9690


In [56]:
plt.figure(figsize=(4.5,4.5))

plt.scatter(test_y,test_p,c='yellowgreen')
plt.text(-3.2,2.6,'$R^2$: %.3f'%r2_score(test_y,test_p),fontsize=16)
plt.text(-3.2,2.1,'Pearson R: %.3f'%pearsonr(test_y,test_p)[0],fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%mean_absolute_error(test_y,test_p),fontsize=16)
plt.plot([y.min(),y.max()],[y.min(),y.max()],c='yellowgreen')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)

plt.tick_params(bottom='on',left='on')
plt.tight_layout()
plt.savefig('./result/test-Grdkit.tif', dpi=600)  
plt.savefig('./result/test-Grdkit.png',dpi = 600)


In [59]:
plt.figure(figsize=(4.5,4.5))

plt.scatter(test_y,test_p,c='yellowgreen')
plt.text(-3.2,2.6,'$R^2$: %.3f'%r2_score(test_y,test_p),fontsize=16)
plt.text(-3.2,2.1,'Pearson R: %.3f'%pearsonr(test_y,test_p)[0],fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%mean_absolute_error(test_y,test_p),fontsize=16)
plt.plot([y.min(),y.max()],[y.min(),y.max()],c='yellowgreen')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)
re_test = np.abs(test_p - test_y) / np.abs(test_y)
accuracy_test = np.mean(re_test <= 0.3) * 100
accuracy_test1 = np.mean(re_test <= 0.2) * 100
plt.text(-0.4, -2.4, f'RE±0.3: {accuracy_test:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold') 
plt.text(-0.4, -2.9, f'RE±0.2: {accuracy_test1:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold')
plt.xticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)
plt.yticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)
plt.tick_params(bottom='on',left='on')
plt.tight_layout()
plt.savefig('./result/test-Grdkit-RE.tif', dpi=600)  
plt.savefig('./result/test-Grdkit-RE.png',dpi = 600)


In [46]:
fig = plt.figure(figsize=(15,40))

plt.barh(importance_desc_names, importance_of_sel_desc, color='lightblue',align='center')
plt.xlabel('Feature Importance Scores',fontsize=19)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tick_params(left='on',bottom='on')
plt.tight_layout()
print()
plt.savefig('./result/Feature Importance Scores-all.png',dpi = 600)

In [47]:
import matplotlib.pyplot as plt
import numpy as np

sorted_indices = np.argsort(importance_of_sel_desc)[-5:]  
top_5_importance = np.array(importance_of_sel_desc)[sorted_indices] 
top_5_names = np.array(importance_desc_names)[sorted_indices]  

fig = plt.figure(figsize=(15, 8)) 

plt.barh(top_5_names, top_5_importance, color='lightblue', align='center')
plt.xlabel('Feature Importance Scores', fontsize=19)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.tick_params(left='on', bottom='on')
plt.tight_layout()
plt.savefig('./result/Feature Importance Scores.png',dpi = 600)
plt.show()

In [48]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
def fixed_selector_transform(X):
    return X[:, sel_index]

fixed_selector = FunctionTransformer(fixed_selector_transform)

pipeline = Pipeline([
    ('scaler', scaler),
    ('fixed_selector', fixed_selector),
    ('model', model)
])

X_full = np.vstack((X.iloc[train_val_indices], X.iloc[full_test_index]))  
y_full = np.concatenate((y_train_val, test_y))

pipeline.fit(X_full, y_full)

import shap
import joblib

X_scaled_full = pipeline.named_steps['scaler'].transform(X_full)
X_selected_full = pipeline.named_steps['fixed_selector'].transform(X_scaled_full)

explainer = shap.TreeExplainer(pipeline.named_steps['model'])
shap_values = explainer.shap_values(X_selected_full)

joblib.dump(pipeline, 'Grdkit.joblib')

['Grdkit.joblib']

In [53]:
y_pred_full = pipeline.named_steps['model'].predict(X_selected_full)

selected_feature_names = importance_desc_names

y_true_aligned = y_full

compound_names = physorg_react_data_df['compound_name'].iloc[train_val_indices].reset_index(drop=True)

df = pd.DataFrame(X_scaled_full[:, sel_index], columns=selected_feature_names)

df.insert(0, 'Compound_Name', compound_names)

df['Predicted_Y'] = y_pred_full

df['True_Y'] = y_true_aligned

missing_true_y = df['True_Y'].isna().sum()
if missing_true_y > 0:
    print(f"发现 {missing_true_y} 个真实值的空白，尝试修复索引对齐...")

df = df.dropna(subset=['True_Y']).reset_index(drop=True)

print(df)

df.to_csv('rdkit-selected_features_and_predicted_y_with_true_y.csv', index=False)

                             Compound_Name  fr_C_O_noCOO  VSA_EState1  \
0                         (pqx)2Irbiim+-ox     -1.021312    -1.021312   
1                            CpIrbpyCl+-ox     -1.117979    -1.117979   
2                             CpIrbpyH+-ox     -1.117979    -1.117979   
3                          Ir(1np)2acac-ox      0.414814     0.414814   
4                           Ir(1np)2dpm-ox      0.803219     0.803219   
5                    Ir(2,4-dFbpy)2acac-ox      1.244828     1.244828   
6                   Ir(2,4-dFppy)2(Pic)-ox      1.274236     1.274236   
7                 Ir(2,4-dMeO-bpy)2acac-ox      0.414814     0.414814   
8                  Ir(2,4-dMeOppy)2acac-ox      0.414814     0.414814   
9        Ir(3,5-dFppy)2(4,4'-dTPS-bpy)+-ox      1.274236     1.274236   
10                         Ir(46dFppy)3-ox      1.274236     1.274236   
11                  Ir(4carbazole-ppy)3-ox     -1.060618    -1.060618   
12                  Ir(4-CFppy)2(PySO3)-ox      1.0

In [50]:
pipeline = joblib.load('Grdkit.joblib')
full_External_index = sorted(list(set(External_index + External_index_db2)))
X_external = X.iloc[full_External_index]
y = physorg_react_data_df['YRed/Ox']
y_external = y.iloc[full_External_index]

ext_pred = pipeline.predict(X_external.to_numpy())
compound_names = physorg_react_data_df['compound_name']
compound_names_external = compound_names.iloc[full_External_index]


In [51]:
results = pd.DataFrame({
    'Compound Name': compound_names_external.values,
    'Predicted Value': ext_pred,
    'Eexp': y_external.values
})

print(results)

                          Compound Name  Predicted Value  Eexp
0                 Ir(dFphtl)2PytlBn+-ox         1.308158  1.37
1                 Ir(dFphtl)2PytlPh+-ox         1.316564  1.38
2                    Ir(dfppz)2Phtz+-ox         1.245729  1.33
3                  Ir(Fphtl)2PytlBn+-ox         1.159666  1.17
4                  Ir(Fphtl)2PytlPh+-ox         1.172079  1.17
5                 Ir(Mebib)(dFppy)CN-ox         0.986026  1.01
6              Ir(Mebib)(mppy)CH3CN+-ox         0.721068  1.06
7                  Ir(Mebib)(mppy)Cl-ox         0.823521  0.68
8               Ir(Mebib)(ppy)CH3CN+-ox         0.731832  1.02
9                   Ir(Mebib)(ppy)CN-ox         0.849630  0.88
10                    Ir(Mebib)ppyBr-ox         0.858313   0.6
11                    Ir(Mebib)ppyCl-ox         0.831181  0.69
12                     Ir(Mebib)ppyI-ox         0.911174  0.71
13                   Ir(Mebip)ppyCl+-ox         0.902768  1.23
14                  Ir(Phbib)(ppy)Cl-ox         0.82866

In [28]:
import pandas as pd

pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)  
pd.set_option('display.max_colwidth', None)  

results = pd.DataFrame({
    'Compound Name': compound_names_external.values,
    'Predicted Value': ext_pred,
    'Eexp': y_external.values
})

print(results)

                          Compound Name  Predicted Value  Eexp
0                 Ir(dFphtl)2PytlBn+-ox         1.308158  1.37
1                 Ir(dFphtl)2PytlPh+-ox         1.316564  1.38
2                    Ir(dfppz)2Phtz+-ox         1.245729  1.33
3                  Ir(Fphtl)2PytlBn+-ox         1.159666  1.17
4                  Ir(Fphtl)2PytlPh+-ox         1.172079  1.17
5                 Ir(Mebib)(dFppy)CN-ox         0.986026  1.01
6              Ir(Mebib)(mppy)CH3CN+-ox         0.721068  1.06
7                  Ir(Mebib)(mppy)Cl-ox         0.823521  0.68
8               Ir(Mebib)(ppy)CH3CN+-ox         0.731832  1.02
9                   Ir(Mebib)(ppy)CN-ox         0.849630  0.88
10                    Ir(Mebib)ppyBr-ox         0.858313   0.6
11                    Ir(Mebib)ppyCl-ox         0.831181  0.69
12                     Ir(Mebib)ppyI-ox         0.911174  0.71
13                   Ir(Mebip)ppyCl+-ox         0.902768  1.23
14                  Ir(Phbib)(ppy)Cl-ox         0.82866

In [ ]:
ext_r2 = r2_score(y_external,ext_pred)
ext_mae = mean_absolute_error(y_external,ext_pred)
ext_pearson_r = pearsonr(y_external,ext_pred)[0]

plt.figure(figsize=(4.5,4.5))

plt.scatter(y_external,ext_pred,c='#efae42')

plt.text(-3.2,2.6,'$R^2$: %.3f'%ext_r2,fontsize=16)
plt.text(-3.2,2.1,'Pearson R: %.3f'%ext_pearson_r,fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%ext_mae,fontsize=16)

plt.plot([y.min(),y.max()],[y.min(),y.max()],c='#efae42')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)
plt.xticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=14)
plt.yticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=14)
plt.tick_params(bottom='on',left='on')

re_ext = np.abs(ext_pred - y_external) / np.abs(y_external)
accuracy_ext = np.mean(re_ext <= 0.3) * 100
accuracy_ext1 = np.mean(re_ext <= 0.2) * 100
plt.text(-0.4, -2.4, f'RE±0.3: {accuracy_ext:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold') 
plt.text(-0.4, -2.9, f'RE±0.2: {accuracy_ext1:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold')

plt.tight_layout()
plt.savefig('./result/external-Grdkit-RE.tif', dpi=600)  
plt.savefig('./result/external-Grdkit-RE.png', dpi=600)

In [60]:
ext_r2 = r2_score(y_external,ext_pred)
ext_mae = mean_absolute_error(y_external,ext_pred)
ext_pearson_r = pearsonr(y_external,ext_pred)[0]

plt.figure(figsize=(4.5,4.5))

plt.scatter(y_external,ext_pred,c='#efae42')

plt.text(-3.2,2.6,'$R^2$: %.3f'%ext_r2,fontsize=16)
plt.text(-3.2,2.1,'Pearson R: %.3f'%ext_pearson_r,fontsize=16)
plt.text(-3.2,1.6,'MAE: %.3f V'%ext_mae,fontsize=16)

plt.plot([y.min(),y.max()],[y.min(),y.max()],c='#efae42')
plt.xlabel('E $_{Exp}$ (V)',fontsize=16)
plt.ylabel('E $_{ML}$ (V)',fontsize=16)
plt.xticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)
plt.yticks([-3, -2, -1, 0,1,2,3],list(map(str,[-3, -2, -1, 0,1,2,3])),fontsize=16)
plt.tick_params(bottom='on',left='on')

re_ext = np.abs(ext_pred - y_external) / np.abs(y_external)
accuracy_ext = np.mean(re_ext <= 0.3) * 100
accuracy_ext1 = np.mean(re_ext <= 0.2) * 100
plt.text(-0.4, -2.4, f'RE±0.3: {accuracy_ext:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold') 
plt.text(-0.4, -2.9, f'RE±0.2: {accuracy_ext1:.2f}%', fontsize=14, fontstyle='italic', fontweight='bold')

plt.tight_layout()
plt.savefig('./result/external-Grdkit-RE.tif', dpi=600)  
plt.savefig('./result/external-Grdkit-RE.png', dpi=600)